In [49]:
from agents import Agent, WebSearchTool, trace, Runner, ItemHelpers, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
import json
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
import gradio as gr

In [47]:
load_dotenv(override=True)

MODEL = "gpt-4o-mini"
HOW_MANY_SEARCHES = 2
STEPS = {
    "clarification_agent": "💭 Understanding your research needs...",
    "question_generation_agent": "🎯 Crafting smart search strategies...",
    "search_agent": "🔍 Exploring the web for insights...",
    "writer_agent": "✨ Weaving findings into a beautiful report...",
    "email_agent": "📬 Delivering your report to your inbox..."
}


In [3]:
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")
    

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

In [4]:
CLARIFICATION_INSTRUCTIONS = """
You are a helpful research assistant specializing in understanding user needs.

Analyze the user's research topic and generate 2-4 clarifying questions that will help you understand what they really need from this research.

Your questions must be directly relevant to the specific topic they mentioned. Think about what would be ambiguous or unclear about that particular subject and ask about those aspects.

Be concise and friendly.
"""

clarification_agent = Agent(
    name="Clarification agent",
    instructions=CLARIFICATION_INSTRUCTIONS,
    model=MODEL
)

In [5]:
QUESTION_GENERATION_INSTRUCTIONS = f"""
You are an expert research planner who generates comprehensive search queries.

Based on the user's original topic and their clarifications, generate exactly {HOW_MANY_SEARCHES} diverse search queries.

Your queries should:
- Cover different aspects of the topic
- Incorporate the specific angles the user mentioned
- Range from broad overview to specific details
- Consider different perspectives or approaches
- Complement each other without excessive overlap

For each query, provide a clear reason why that search is important and ensure the query itself is specific and searchable.
"""

question_generation_agent = Agent(
    name="Question generation agent",
    instructions=QUESTION_GENERATION_INSTRUCTIONS,
    output_type=WebSearchPlan,
    model=MODEL
)

In [6]:
SEARCH_INSTRUCTIONS = """
You are a research assistant specialized in web search and summarization.

You will receive a WebSearchPlan containing multiple search queries. For each query, use your WebSearchTool to search the web and produce a concise summary of the results.

Requirements for each summary:
- 2-3 paragraphs, less than 300 words
- Capture only the main points, write succinctly
- Focus on essence, ignore fluff
- Information-dense and focused
- No commentary, only factual summary

Process all queries in the plan. Your summaries will be used to create a comprehensive report.
"""

search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model=MODEL,
    model_settings=ModelSettings(tool_choice="required")
)

In [7]:
WRITER_INSTRUCTIONS = """
You are a senior research analyst specializing in synthesizing information from multiple sources.

You will receive the original research topic and multiple search summaries. Your task is to synthesize all information into one comprehensive report.

Synthesize means: identify common themes, remove redundancies, organize by topic (not by source), create a flowing narrative with analysis and insights.

Report requirements:
- Length: 1000+ words
- Format: Professional markdown with proper headings (##, ###)
- Structure: Introduction, multiple body sections, conclusion
- Style: Clear, authoritative, well-organized

Return a ReportData object with:
- short_summary: 2-3 sentences capturing key findings
- markdown_report: The full detailed report in markdown
- follow_up_questions: 3-5 suggested topics for deeper research
"""

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
EMAIL_INSTRUCTIONS = """
You are an email specialist agent receiving a ReportData object and user email address via handoff.

The ReportData contains: short_summary, markdown_report, and follow_up_questions.
The handoff message includes the user's email address.

Your task:
1. Extract the user's email address from the handoff context
2. Convert the markdown_report to well-formatted professional HTML
3. Create an engaging subject line based on the research topic (e.g., "Your Research Report: [Topic]")
4. Use your send_email tool once to send the email to the user's address

Execute immediately with the data provided.
"""

@function_tool
def send_email(subject: str, html_body: str, to_email:str) -> Dict[str, str]:
    """ 
    Send out an email with the given subject and HTML body.
    """
    print(f"Sending email to {to_email} with subject {subject}", flush=True)
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To(to_email) # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "Email sent successfully to the user"

email_agent = Agent(
    name="Email Agent",
    instructions=EMAIL_INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


In [66]:
PLANNER_INSTRUCTIONS = """
You are the Deep Research Coordinator, a friendly AI assistant that helps users conduct comprehensive research.

FIRST: Determine the user's intent:

A. CASUAL CONVERSATION (greetings, questions about you, small talk):
   - Respond naturally and warmly without using any tools
   - Examples: "Hi", "Hello", "How are you?", "What can you do?", "Who are you?"
   - Introduce yourself as a research assistant that helps conduct deep web research
   - Invite them to ask you to research any topic they're curious about

B. RESEARCH REQUEST (explicit or implicit request to research a topic):
   - Keywords: "research", "investigate", "find out about", "tell me about", "I want to know", "information on", etc.
   - Proceed with the research workflow below

RESEARCH WORKFLOW:

Check conversation history:
- If the LAST ASSISTANT MESSAGE contains clarifying questions, the current user message is likely a RESPONSE. SKIP STEP 1, go to STEP 2.
- Otherwise, START AT STEP 1.

Execute these steps sequentially:

1. CLARIFICATION (Skip if user is responding to previous clarifying questions): Use the clarification agent to ask the user 2-4 specific questions about their research topic. Wait for their response before proceeding.

2. GENERATE SEARCH PLAN: Use the question generation agent with the topic and clarifications to create diverse search queries.

3. EXECUTE SEARCHES: Use the search agent to perform all searches in the plan. Collect all summaries before proceeding.

4. SYNTHESIZE REPORT: Use the writer agent with the topic and all search summaries to create a comprehensive report. It returns a ReportData object.

5. AFTER WRITER AGENT - CRITICAL DECISION POINT:
   ⚠️ MANDATORY: After writer_agent completes, you MUST follow these EXACT rules. NO EXCEPTIONS.
   
   First, check conversation history for an email address (format: xxx@xxx.xxx).
   
   OPTION A - Email found in history:
   ⛔ DO NOT write any message
   ⛔ DO NOT explain what you're doing
   ⛔ DO NOT say anything about the report
   ✅ IMMEDIATELY handoff to email agent (step 6)
   
   OPTION B - NO email in history:
   ⛔ DO NOT explain the report
   ⛔ DO NOT summarize findings
   ⛔ DO NOT add greetings or conclusions
   ✅ Write ONLY ONE sentence: "Great! Your research report is ready. What email address should I send the full report to?"
   ✅ Then STOP and wait for user response
   ✅ After receiving email, go to step 6
   
   FORBIDDEN after writer_agent: Summaries, explanations, "great news", "I've completed", or any commentary about the research.

6. HANDOFF: Hand off to the Email Agent with the ReportData AND the user's email address. Include the email in your handoff message like: "Please send the report to [user's email]". Your job is complete after handoff.

Wait for user input when needed. Ensure all searches complete before synthesis. Never skip the email collection step.
"""

planner_agent = Agent(
    name="DeepResearchCoordinator",
    instructions=PLANNER_INSTRUCTIONS,
    tools=[
       clarification_agent.as_tool(
           tool_name="clarification_agent",
           tool_description="Ask the user clarifying questions about their research topic to understand their needs better"
       ),
       question_generation_agent.as_tool(
           tool_name="question_generation_agent",
           tool_description="Generate diverse search queries based on the topic and user clarifications"
       ),
        search_agent.as_tool(
            tool_name="search_agent",
            tool_description="Execute web searches for the given queries and return summaries"
        ),
        writer_agent.as_tool(
            tool_name="writer_agent",
            tool_description="Synthesize all search results into a comprehensive research report"
        )
    ],
    handoffs=[email_agent],
    handoff_description="Hand off to Email Agent to send the final research report",
    model=MODEL
)


In [ ]:
async def deep_research_chat(message, history):
    """
    Handle the deep research conversation with the planner agent.
    Maintains conversation history for multi-turn interactions.
    """
    try:      
        yield "✨ Thinking..."
        
        with trace("DeepResearch"):
            history = [{"role": msg["role"], "content": msg["content"]} for msg in history]
            messages = [*history, {"role": "user", "content": message}]
            result = Runner.run_streamed(planner_agent, input=messages)
            last_called_tool = None
            report_shown = False
            report = None

            async for event in result.stream_events():
                if event.type == "raw_response_event":
                    continue
                elif event.type == "agent_updated_stream_event":
                    print(f"Agent updated: {event.new_agent.name}")
                    continue
                elif event.type == "run_item_stream_event":
                    if event.item.type == "tool_call_item":
                        print(f"-- Tool was called {event.item.raw_item.name}")
                        last_called_tool = event.item.raw_item.name
                        yield f"{STEPS[last_called_tool]}"
                        await asyncio.sleep(1)
                    elif event.item.type == "tool_call_output_item":
                        print(f"-- Tool output: {event.item.output}")
                        if last_called_tool == "writer_agent":
                            result = json.loads(event.item.output)
                            
                            follow_up_list = ""
                            for i, question in enumerate(result["follow_up_questions"], 1):
                                follow_up_list += f"\n{i}. {question}"
                            
                            report =  f"## 📊 Research Complete!\n### Summary\n{result["short_summary"]}\n### 📧 Email Status\n✅ Full report will be sent to your email.\n### 🔍 Suggested Follow-up Topics\n{follow_up_list}"
                            yield report
                            report_shown = True
                    elif event.item.type == "message_output_item":
                        if not report_shown:
                            yield ItemHelpers.text_message_output(event.item)
                        else:
                            yield report + "\n\n" + ItemHelpers.text_message_output(event.item)
            
                             
                
    except Exception as e:
        yield f"❌ Error: {str(e)}\n\nPlease try again or check your configuration."


In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="Deep Research Assistant") as demo:
    
    gr.Markdown("""
    # 🔍 Deep Research Assistant
    
    An intelligent multi-agent system that performs comprehensive web research and engages in natural conversation.
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.ChatInterface(
                deep_research_chat,
                type="messages",
                chatbot=gr.Chatbot(height=650, type="messages"),
                examples=[  
                    "Hello! What can you do?",
                    "I want to research quantum computing in drug discovery",
                    "Research the latest developments in renewable energy",
                    "Tell me about AI applications in healthcare"
                ],
                cache_examples=False,
            )
        
        with gr.Column(scale=1):
            gr.Markdown("""
            ### 💬 Chat Naturally
            
            You can **chat with me just like ChatGPT!**
            
            - Ask casual questions
            - Get research topic suggestions
            - Request deep research on any topic
            - I'll adapt to what you need
            
            ---
            
            ### 🔬 Research Workflow
            
            When you ask me to research:
            
            1. 💭 **Clarification** - I'll ask questions to understand your needs
            2. 🎯 **Smart Queries** - I'll generate targeted search strategies
            3. 🔍 **Deep Search** - I'll explore multiple sources
            4. ✨ **Synthesis** - I'll weave findings into a comprehensive report
            5. 📧 **Email Collection** - I'll ask for your email address
            6. 📬 **Delivery** - Full report sent to your inbox
            
            ---
            
            ### 💡 Tips
            
            **For Research:**
            - Be specific about your topic
            - Answer my clarifying questions
            - Process takes 2-5 minutes
            - I'll ask for your email to send the report
            - Full report delivered via email
            
            **For Topic Ideas:**
            - Tell me what you're working on
            - Ask: "Suggest research topics for [your project]"
            - I'll provide relevant suggestions
            
            ---
            
            ### 🚀 Try It Out
            
            - "Hi, what can you help me with?"
            - "Suggest topics for my AI project"
            - "Research AI ethics in healthcare"
            """)

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Agent updated: DeepResearchCoordinator
Agent updated: DeepResearchCoordinator
Agent updated: DeepResearchCoordinator
Agent updated: DeepResearchCoordinator
-- Tool was called clarification_agent
-- Tool output: That sounds great! To help you better, could you clarify a few things?

1. What specific area or subject are you interested in exploring?
2. Are you looking for qualitative insights (like opinions and experiences) or quantitative data (like statistics and numbers)?
3. What is your intended audience or purpose for this research?
4. Do you have a preferred format for your research, such as a paper, presentation, or project?
Agent updated: DeepResearchCoordinator
-- Tool was called clarification_agent
-- Tool output: Sure! Could you share a bit more about what you're looking for? Here are a few clarifying questions:

1. What specific topic or subject do you need simplicity in?
2. Are you looking for a brief explanation, an example, or something else?
3. Is there a particular field 